In [15]:
import logging


def init_logger(file_name, log_level=None):
    if not log_level:
        log_level = "INFO"
    logging.basicConfig()
    logger = logging.getLogger(file_name)
    logger.setLevel(getattr(logging, log_level))
    return logger

In [16]:
import awswrangler as wr
import boto3
import pandas as pd
from collections import defaultdict
import json

In [17]:
logger = init_logger(__name__)

In [18]:
myProfile = 'jvalenzuela-tesis'
mySession = boto3.session.Session(profile_name=myProfile)
ssmClient = mySession.client('ssm', region_name = 'us-east-1')

In [19]:
rawBucket = ssmClient.get_parameter(Name='/Master/S3/CentralBucket')['Parameter']['Value']
stageBucket = ssmClient.get_parameter(Name='/Master/S3/StageBucket')['Parameter']['Value']

In [20]:
def getObjectList(s3PathKey):
    return wr.s3.list_objects(s3PathKey, boto3_session=mySession)

In [21]:
def s3FilesProcessing(csvObjects, dataset, s3OutputPath): 
    df_array = []
    filepartition = 'file_name'
    for f_name in csvObjects:
        logging.info(f"Reading s3 File: {f_name}")
        df = wr.s3.read_json(path=f_name, 
                         orient = 'records',
                         boto3_session=mySession)
        
        if dataset == 'three-sixty':
            filenameValue = f_name.split('/')[-1].replace('.json', '')
            df[filepartition] = filenameValue
        
        if dataset == 'events': 
            filepartition = 'match_id'
            filenameValue = f_name.split('/')[-1].replace('.json', '')
            df[filepartition] = filenameValue
            
            df['tactics'] = df['tactics'].apply(json.dumps)
            df['type'] = df['type'].apply(json.dumps)
            df['possession_team'] = df['possession_team'].apply(json.dumps)
            df['play_pattern'] = df['play_pattern'].apply(json.dumps)
            df['team'] = df['team'].apply(json.dumps)
            
            dftactics = pd.json_normalize(df.tactics.apply(json.loads)).add_prefix('tactics.')
            dftype = pd.json_normalize(df.type.apply(json.loads)).add_prefix('type.')
            dfpossession_team = pd.json_normalize(df.possession_team.apply(json.loads)).add_prefix('possession_team.')
            dfplay_pattern = pd.json_normalize(df.play_pattern.apply(json.loads)).add_prefix('play_pattern.')
            dfteam = pd.json_normalize(df.team.apply(json.loads)).add_prefix('team.')
            
            df = pd.concat([df, dftactics, dftype, dfpossession_team, dfplay_pattern, dfteam], axis=1)
     
    #df_array.append(df)
    #logging.info(f"Concatination of Dataframe Array")
    #df_concat = pd.concat(df_array)
    
        wr.s3.to_parquet(
            df=df,
            path= s3OutputPath,
            dataset = True,
            mode = 'append',
            partition_cols = [filepartition],
            boto3_session=mySession
        )

In [24]:
event = {"dataset":"three-sixty"}
context = None
def lambda_handler(event, context):
    try:
        dataset = event['dataset']
        s3RawPath = f"s3://{rawBucket}/datalake/{dataset}/"
        print(f"Using tha following Raw bucket Path: {s3RawPath}")
        s3StagePath = f"s3://{stageBucket}/datalake/{dataset}/"
        print(f"Using tha following Stage bucket Path: {s3StagePath}")        
        ObjectList = getObjectList(s3RawPath)
        s3FilesProcessing(ObjectList, dataset, s3StagePath)
    except Exception as e: 
        print(f"Error on LambdaHandler has ocurred: {str(e)}")
    

In [25]:
lambda_handler(event, context)

Using tha following Raw bucket Path: s3://soccer-datalake-dev-us-east-1-829547505377-raw/datalake/three-sixty/
Using tha following Stage bucket Path: s3://soccer-datalake-dev-us-east-1-829547505377-stage/datalake/three-sixty/


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be return